### 8. Работа с транзакциями

Транзакции позволяют выполнять группу операций в базе данных как единое целое, что обеспечивает целостность данных. В Django ORM управление транзакциями осуществляется с помощью контекста `atomic`, который гарантирует, что все операции внутри блока либо выполняются, либо не выполняются вовсе в случае ошибки. Это особенно полезно при работе с множеством связанных операций, чтобы избежать частичных изменений данных.


#### 1. Основы транзакций

**Транзакция** — это последовательность операций, которые должны быть выполнены полностью или не выполнены вовсе. Например, если мы переводим деньги между двумя счетами, необходимо убедиться, что сумма списана с одного счета и добавлена на другой. Если одно из этих действий не удается, транзакция должна быть отменена.

##### Пример сценария с транзакциями

Рассмотрим пример, где мы хотим перевести деньги с одного аккаунта на другой. Если возникнет ошибка во время выполнения одной из операций, мы хотим отменить все изменения.


In [ ]:
from django.db import transaction

def transfer_funds(sender_account, receiver_account, amount):
    with transaction.atomic():
        # Убедимся, что на счете достаточно средств
        if sender_account.balance < amount:
            raise ValueError("Недостаточно средств на счете.")
        
        # Сначала списываем средства с отправителя
        sender_account.balance -= amount
        sender_account.save()

        # Теперь добавляем средства получателю
        receiver_account.balance += amount
        receiver_account.save()

В этом примере все операции по переводу денег заключены в блок `with transaction.atomic():`. Если на счете отправителя недостаточно средств, возникает ошибка, и все изменения откатываются. Это гарантирует, что состояние базы данных останется целостным и непротиворечивым.

#### 2. Использование `atomic`

Метод `transaction.atomic()` может использоваться как в контексте (как в приведенном выше примере), так и как декоратор для функций. Вот как можно использовать его в качестве декоратора:

In [ ]:
from django.db import transaction

@transaction.atomic
def create_order(user, cart):
    # Создание нового заказа
    order = Order.objects.create(user=user)
    
    for item in cart.items:
        order_item = OrderItem.objects.create(order=order, product=item.product, quantity=item.quantity)
        
        # Уменьшаем количество товара на складе
        item.product.stock -= item.quantity
        item.product.save()

В этом примере, если что-то пойдет не так при создании заказа или уменьшении запаса, все изменения будут отменены.

#### 3. Уровни изоляции

Django позволяет задавать уровень изоляции транзакций, используя параметр `isolation_level`. Это определяет, как транзакции взаимодействуют друг с другом.

Пример:

In [ ]:
from django.db import transaction

@transaction.atomic(isolation_level='READ COMMITTED')
def update_account_balance(account, new_balance):
    account.balance = new_balance
    account.save()

Наиболее распространенные уровни изоляции:
- **READ UNCOMMITTED**: позволяет видеть изменения в других транзакциях, даже если они еще не подтверждены.
- **READ COMMITTED**: гарантирует, что транзакция видит только те данные, которые были зафиксированы до начала ее выполнения.
- **REPEATABLE READ**: гарантирует, что транзакция будет видеть одни и те же данные при каждом обращении к базе данных.
- **SERIALIZABLE**: самый строгий уровень, который обеспечивает полную изоляцию и предотвращает аномалии, но может приводить к блокировкам.


#### 4. Исключения и откат транзакций

Если возникает исключение внутри блока `atomic`, все изменения, выполненные в рамках этой транзакции, будут отменены. Это позволяет легко обрабатывать ошибки и поддерживать целостность данных.

Пример:

In [ ]:
from django.db import transaction

def safe_transfer_funds(sender_account, receiver_account, amount):
    try:
        with transaction.atomic():
            transfer_funds(sender_account, receiver_account, amount)
    except Exception as e:
        print(f"Произошла ошибка: {e}. Транзакция отменена.")

Если в функции `transfer_funds` произойдет ошибка, она будет поймана, и все изменения будут отменены.

#### Заключение

Работа с транзакциями в Django ORM с использованием `transaction.atomic()` является важной частью обеспечения целостности и согласованности данных. Понимание принципов работы транзакций, уровней изоляции и обработки исключений позволяет разработчикам создавать надежные и безопасные приложения. Студенты научатся правильно использовать транзакции в своих проектах, что поможет избежать потенциальных проблем с данными при выполнении сложных операций.